In [1]:
import pickle
import numpy as np

In [2]:
file = "../DeepMoji-master/data/PsychExp/raw.pickle"

In [3]:
data = pickle.load(open(file,'rb'),encoding='latin1')

In [4]:
label = data['info']

texts = data['texts']

val_index = data['val_ind']

test_index = data['test_ind']

train_index = data['train_ind']

In [5]:
def glove_vec(path):
    import numpy as np
    with open(path,'r',encoding='utf-8') as f:
        words = set()
        word_vec = {}
        for line in f:
            line = line.strip().split()
            key = line[0]
            words.add(key)
            val = np.array(line[1:],dtype=np.float64)
            word_vec[key] = val
            
        # Dict (Vocab) --> set
        word2idx = {}
        idx2word = {}
        
        for i,w in enumerate(sorted(words)):
            word2idx[w] = i
            idx2word[i] = w
            
    return word_vec,word2idx,idx2word

In [6]:
word_vec,word2idx,idx2word = glove_vec('../glove.6B/glove.6B.50d.txt')
vocab_size = len(word_vec)

In [17]:
from nltk import word_tokenize

nodes = 128

In [18]:
def get_XY_data(Index):
    X = np.zeros((len(Index) , nodes ))
    Y = np.zeros((len(Index) , 7))
    
    zer = np.zeros((50 ,))
    for i ,index in enumerate(Index):
        line = word_tokenize(texts[index].lower())
        
        #print(index)
        Y[i] = label[index]['label']
        for j , word  in  enumerate(line[:nodes]):
             X[i , j] =  word2idx.get(word , 0)
        #print(X[i])
    return X ,Y


In [19]:
corpus_len = len(word2idx) + 1
embed_dim = 50

embed_matrix = np.zeros((corpus_len,embed_dim))
for word, index in word2idx.items():
    embed_matrix[index,:] = word_vec[word]


In [20]:
X_train , Y_train  = get_XY_data(train_index)
print(X_train.shape , Y_train.shape)

#X_test , Y_test  = get_XY_data(test_index)
#print(X_test.shape , Y_test.shape)

X_val , Y_val  = get_XY_data(val_index)
print(X_val.shape , Y_val.shape)


(900, 128) (900, 7)
(100, 128) (100, 7)


In [88]:
from keras.layers import Embedding , Flatten ,Dense , Bidirectional ,LSTM
from keras.models import Sequential
from keras.layers import BatchNormalization

In [89]:
model = Sequential()
model.add(Embedding(vocab_size + 1,50, input_length=nodes , trainable = False ))
model.add(Bidirectional(LSTM(nodes ,  return_sequences = True  , recurrent_dropout = 0.2 )))
model.add(Flatten())
model.add(Dense(7, activation='softmax'))

In [90]:
model.layers[0].set_weights([embed_matrix])

In [91]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
print(model.summary())

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 128, 50)           20000050  
_________________________________________________________________
bidirectional_1 (Bidirection (None, 128, 256)          183296    
_________________________________________________________________
flatten_1 (Flatten)          (None, 32768)             0         
_________________________________________________________________
dense_1 (Dense)              (None, 7)                 229383    
Total params: 20,412,729
Trainable params: 412,679
Non-trainable params: 20,000,050
_________________________________________________________________
None


In [92]:
history = model.fit(X_train , Y_train , batch_size = 16 , epochs=7 , validation_data=(X_val,Y_val))

Epoch 1/7
57/57 [==============================] - 12s 218ms/step - loss: 2.4369 - accuracy: 0.1600 - val_loss: 1.8916 - val_accuracy: 0.2300
Epoch 2/7
57/57 [==============================] - 12s 210ms/step - loss: 1.7409 - accuracy: 0.3189 - val_loss: 1.8217 - val_accuracy: 0.2500
Epoch 3/7
57/57 [==============================] - 12s 208ms/step - loss: 1.5526 - accuracy: 0.3989 - val_loss: 1.8435 - val_accuracy: 0.2600
Epoch 4/7
57/57 [==============================] - 12s 208ms/step - loss: 1.3654 - accuracy: 0.5000 - val_loss: 1.7078 - val_accuracy: 0.3500
Epoch 5/7
57/57 [==============================] - 12s 211ms/step - loss: 1.2497 - accuracy: 0.5644 - val_loss: 1.7487 - val_accuracy: 0.3400
Epoch 6/7
57/57 [==============================] - 12s 209ms/step - loss: 1.0900 - accuracy: 0.6444 - val_loss: 1.8301 - val_accuracy: 0.3600
Epoch 7/7
57/57 [==============================] - 12s 210ms/step - loss: 0.9649 - accuracy: 0.6744 - val_loss: 1.8851 - val_accuracy: 0.3700


In [79]:
model.save("model.h5")